# Equation of state (EOS)

For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Example: Silicon

### Prepare the inputs and submit the workflow



In [ ]:
from ase.build import bulk
from ase import Atoms
from aiida import load_profile
from workgraph_collections.ase.espresso.eos import EosWorkGraph

load_profile()


# create input structure
atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate quantum-espresso-7.4
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "control": {"calculation": "scf"},
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
pw_inputs = {
    "command": "mpirun -np 2 pw.x",
    "input_data": input_data,
    "kpts": [4, 4, 4],
    "pseudopotentials": pseudopotentials,
    "pseudo_dir": pseudo_dir,
    "metadata": metadata,
    "computer": "localhost",
}
#------------------------- Set the inputs -------------------------
wg = EosWorkGraph.build(atoms=atoms,
                   scales=[0.95, 0.98, 1.0, 1.02, 1.05],
                   pw_inputs=pw_inputs,
                   )
wg.to_html()

Run the calculation


In [2]:
wg.run()


10/08/2025 11:48:23 AM <2554077> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201874|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[201874|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator
10/08/2025 11:48:24 AM <2554077> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [201874|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 201876
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[201874|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 201876
10/08/2025 11:48:33 AM <2554077> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [WARNING] output parser returned exit code<326>: Function execution failed.
wall_time_seconds found in both dictionaries with different values: 4.6 vs. 4.594454050064087
Traceback (most recent call last):
  File "<stdin>", line 41, in main
  File "/hom

{}

Print the results

In [3]:
#------------------------- Print the output -------------------------
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wg.outputs.result.value))


TypeError: str.format() argument after ** must be a mapping, not NoneType